In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
train_df = pd.read_csv("C:\\Users\\SaiJyosthnaYenumula\\OneDrive - Blend 360\\Desktop\\COHORT10\\coding challenge\\Coding challenge 3 DS\\train.csv")
test_df = pd.read_csv("C:\\Users\\SaiJyosthnaYenumula\\OneDrive - Blend 360\\Desktop\\COHORT10\\coding challenge\\Coding challenge 3 DS\\test.csv")

In [4]:
train_df = train_df.rename(columns={'Store_no.': 'Store'})
test_df = test_df.rename(columns={'Store_no.': 'Store'})

In [5]:
stores=pd.read_csv("C:\\Users\\SaiJyosthnaYenumula\\OneDrive - Blend 360\\Desktop\\COHORT10\\coding challenge\\Coding challenge 3 DS\\stores.csv")

In [6]:
train_df = pd.merge(train_df, stores, on='Store', how='inner')
test_df = pd.merge(test_df, stores, on='Store', how='inner')

In [7]:
train_df['Date'] = pd.to_datetime(train_df['Date'], format='%d-%m-%Y')
# extract year, week, and month into new columns
train_df['year'], train_df['week'], _ = zip(*train_df['Date'].apply(lambda x: x.isocalendar()))
train_df['month'] = train_df['Date'].dt.month

In [8]:
test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d-%m-%Y')
# extract year, week, and month into new columns
test_df['year'], test_df['week'], _ = zip(*test_df['Date'].apply(lambda x: x.isocalendar()))
test_df['month'] = test_df['Date'].dt.month

In [9]:
train_df.drop('Date',axis=1,inplace=True)
test_df.drop('Date',axis=1,inplace=True)

In [10]:
train_df = pd.get_dummies(train_df, columns=['Type'],drop_first=True)
#label_encoder = LabelEncoder()
#train_df['Store'] = label_encoder.fit_transform(train_df['Store'])
#train_df['Department'] = label_encoder.fit_transform(train_df['Department'])

In [11]:
test_df = pd.get_dummies(test_df, columns=['Type'], drop_first=True)
#test_df['Store'] = label_encoder.fit_transform(test_df['Store'])
#test_df['Department'] = label_encoder.fit_transform(test_df['Department'])

In [12]:
train_df[train_df['Weekly_sales']<0]['Weekly_sales'].value_counts()

-15.00     25
-5.00      14
-2.00      13
-4.00       9
-12.00      9
           ..
-5.12       1
-42.78      1
-465.08     1
-90.00      1
-0.24       1
Name: Weekly_sales, Length: 408, dtype: int64

In [13]:
train_df=train_df[train_df['Weekly_sales']>0]

In [14]:
# split the train data into train and validation sets
from sklearn.model_selection import train_test_split

X = train_df.drop(['Weekly_sales'], axis=1)
y = train_df['Weekly_sales']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

y_pred = lr_model.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred)
print(f"MAPE for Linear Regression: {mape}")

# random forest regression model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred)
print(f"MAPE for Random Forest Regression: {mape}")

# gradient boosting regression model
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred)
print(f"MAPE for Gradient Boosting Regression: {mape}")

MAPE for Linear Regression: 133.32884896100617
MAPE for Random Forest Regression: 0.654504155143873
MAPE for Gradient Boosting Regression: 73.33677686149912


In [16]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [17]:
from xgboost import XGBRegressor as xgb
x = xgb(random_state=42)
x.fit(X_train, y_train)

y_pred = x.predict(X_val)
mape = mean_absolute_percentage_error(y_val, y_pred)
print(f"MAPE for XGBoosting Regression: {mape}")

MAPE for XGBoosting Regression: 26.326513378238793


In [18]:
test_pred=rf_model.predict(test_df)
test_df['Weekly_sales']=test_pred

In [19]:
test_df

,Store,Department,Holiday_ind,Size,year,week,month,Type_B,Type_C,Weekly_sales
0,1,1,0,151315,2018,31,8,0,0,16383.8111
1,1,1,0,151315,2018,32,8,0,0,15882.6427
2,1,1,0,151315,2018,33,8,0,0,15888.7380
3,1,1,0,151315,2018,34,8,0,0,15852.0714
4,1,1,0,151315,2018,35,8,0,0,15931.9417
...,...,...,...,...,...,...,...,...,...,...
22100,30,83,0,118221,2018,39,9,1,0,472.3450
22101,30,83,0,118221,2018,40,10,1,0,524.2162
22102,30,83,0,118221,2018,41,10,1,0,533.6894
22103,30,83,0,118221,2018,42,10,1,0,581.9308


In [24]:
result_df=pd.DataFrame(test_df,columns=["Store","Department","week","Weekly_sales"])
result_df.to_csv("predictions.csv",index=False)